In [1]:
import os
os.chdir("../")

# Configuration Manager

In [8]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir : Path
    source_URL : str
    local_data_file : Path
    unzip_dir : Path

In [ ]:
from mlproject.constants import *
from mlproject.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(self, 
                 config_filepath = CONFIG_FILE_PATH, 
                 params_filepath = PARAMS_FILE_PATH, 
                 schema_filepath = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config

# Data Ingestion Component

(How it will ingest the data)

In [10]:
import os
import zipfile
from mlproject import logger
import urllib.request as request
from mlproject.utils.common import get_size

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    # download data from URL
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(url=self.config.source_URL, filename=self.config.local_data_file)
            logger.info(f"{filename} Downloaded! with following info:\n{headers}")
        else:
            logger.info(f"File Already Exists! {get_size(Path(self.config.local_data_file))}")

    # extract downloaded zip file
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None 
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

# Updating the Pipeline

In [ ]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-06-27 11:16:52,110: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-06-27 11:16:52,113: INFO: common: yaml file: params.yaml loaded successfully]
[2025-06-27 11:16:52,117: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-06-27 11:16:52,120: INFO: common: created directory at: artifacts]
[2025-06-27 11:16:52,121: INFO: common: created directory at: artifacts/data_ingestion]
[2025-06-27 11:16:52,122: INFO: 666142431: File Already Exists! ~ 23 KB]
